

# Amazon Bedrock GuardrailsでStrands Agentsを保護する

## 概要
この例では、[Amazon Bedrock Guardrails](https://aws.amazon.com/bedrock/guardrails/)を使用して基盤となるAmazon Bedrock LLMモデルを保護する、最初のStrands Agentを作成する方法をガイドします。

Amazon Bedrock Guardrailsは、生成AIアプリケーションを安全に大規模構築するための設定可能な保護機能を提供します。Amazon Bedrockでサポートされているファウンデーションモデル（FM）、微調整されたモデル、Amazon Bedrock外部でホストされているモデルを含む幅広いファウンデーションモデルで使用される一貫した標準的なアプローチにより、Guardrailsは業界をリードする安全保護を提供します。

Strands Agentsでは、Amazon Bedrock GuardrailsをAmazon Bedrockモデルに直接追加できます。Amazon Bedrockモデルを使用していない場合は、[Apply Guardrail API](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-use-independent-api.html)を使用して任意のモデルを保護できます。この場合、学習したばかりの高度な処理機能を使用してパイプラインを構築する必要があります。


## エージェントの詳細
<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                        |
|--------------------|-------------------------------------------|
|使用するAWSサービス   |Amazon Bedrock Guardrails                 |
|作成するカスタムツール|get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile|
|エージェント構造     |シングルエージェントアーキテクチャ              |

</div>


## アーキテクチャ

<div style="text-align:left">
    <img src="images/architecture.png" width="85%" />
</div>

## 主な機能
* **シングルエージェントアーキテクチャ**: この例では、組み込みツールとカスタムツールの両方とやり取りする単一のエージェントを作成します
* **カスタムツール**: 独自のツールを作成する方法を学びます
* **基盤となるLLMとしてのAmazon Bedrockモデル**: 基盤となるLLMモデルとして、Amazon BedrockからAnthropic Claude 3.7を使用します
* **基盤となる保護機能としてのAmazon Bedrock Guardrails**: Amazon Bedrock Guardrailsを使用してエージェントアプリケーションを保護します

## セットアップと前提条件

### 前提条件
* Python 3.10以上
* AWSアカウント
* Amazon BedrockでAnthropic Claude 3.7が有効化されていること
* Amazon Bedrock Guardrailsを作成する権限を持つIAMロール

それでは、Strands Agentに必要なパッケージをインストールしましょう

In [ ]:
!pip install -U -r requirements.txt

### 依存パッケージのインポート

それでは、依存パッケージをインポートしましょう

In [ ]:
import json
import boto3
import os
from strands import Agent, tool
from strands.models import BedrockModel
from customer_profile_tools import get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile
from customer_profiles import CustomerProfileManager, generate_synthetic_profiles

### データの生成

この例のために合成データを生成しましょう。実際には、エージェントは既存の顧客データベースに接続しますが、この例ではJSONファイルを使用します

In [ ]:
 # Generate synthetic profiles if needed
profile_manager = CustomerProfileManager()
if not os.path.exists("customer_profiles.json"):
    print("Generating synthetic customer profiles")
    profiles = generate_synthetic_profiles(10)
    print(f"Generated {len(profiles)} synthetic customer profiles")
else:
    print("Using existing customer profiles")
    profiles = list(profile_manager.profiles.values())
    print(f"Loaded {len(profiles)} customer profiles")

# Display sample profile for reference
if profiles:
    sample_profile = profiles[0]
    print(f"\nSample Profile: {sample_profile.name}")
    print(f"Email: {sample_profile.email}")
    print(f"Customer ID: {sample_profile.customer_id}")
    print(f"Location: {sample_profile.state}, {sample_profile.country}")
    print(f"Purchases: {len(sample_profile.purchase_history)}")
    print(f"Support Tickets: {len(sample_profile.support_tickets)}")

## Amazon Bedrock Guardrailの作成

それでは、金融アドバイスの提供を防ぎ、顧客向けアプリケーションの一般的な課題をフィルタリングするAmazon Bedrock Guardrailを作成しましょう。

guardrailを作成するには、まずAmazon Bedrockクライアントを設定する必要があります：

In [ ]:
# Configure AWS clients
bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

これで、`create_guardrail`メソッドを使用して、アプリケーションで使用される`no-investment-advice` guardrailを作成できます。この例では、guardrailにはエージェントが信託アドバイスを提供することをブロックする`Fiduciary Advice`のトピック拒否が含まれます。guardrailには、不適切なコンテンツをフィルタリングするための基本的なコンテンツポリシーと、特定の事前定義された単語を検出するための単語ポリシー設定も含まれます。

In [ ]:
# Create a Bedrock guardrail
response = bedrock_client.create_guardrail(
    name='no-investment-advice',
    description='Prevents the model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    blockedInputMessaging='I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.',
    blockedOutputsMessaging='I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including financial, or restricted details.',
)

# Print the response to get the guardrail ID
print("Guardrail ID:", response.get('guardrailId'))
print("Guardrail ARN:", response.get('guardrailArn'))

# Store the guardrail ID for later use


In [ ]:
guardrail_id = response.get('guardrailId')
guardrail_version = "DRAFT"  # Initial version is always 1

### guardrailの直接テスト

guardrailが期待どおりに機能することを確認するために、`test_guardrail`サポート関数を作成します。この関数は`apply_guardrail`メソッドを使用して入力テキストを保護し、guardrailによって取られたアクションに関する情報を提供します

In [ ]:
# Test function to check if input/output is blocked by guardrail
def test_guardrail(text, source_type='INPUT'):
      response = bedrock_runtime.apply_guardrail(
          guardrailIdentifier=guardrail_id,
          guardrailVersion=guardrail_version,
          source=source_type,  # can be 'INPUT' or 'OUTPUT'
          content=[{"text": {"text": text}}]
      )

      # New response format uses different fields
      print(f"Action: {response.get('action')}")
      print(f"Action Reason: {response.get('actionReason', 'None')}")

      # Check if content was blocked
      is_blocked = response.get('action') == 'GUARDRAIL_INTERVENED'
      print(f"Content {source_type} blocked: {is_blocked}")

      if is_blocked:
          # Print topic policies that were triggered
          assessments = response.get('assessments', [])
          if assessments and 'topicPolicy' in assessments[0]:
              print("Blocked topics:", [topic.get('name') for topic in
  assessments[0]['topicPolicy'].get('topics', [])
                                       if topic.get('action') == 'BLOCKED'])

          # Print the modified output if available
          if 'outputs' in response and response['outputs']:
              print("Modified content:", response['outputs'][0].get('text', 'None'))

      return response

# Test some safe input
print("Testing safe input:")
test_guardrail("Tell me about general financial literacy concepts.")

# Test input that should be blocked
print("\nTesting input that should be blocked:")
test_guardrail("What stocks should I invest in for my retirement?")

## Strands Agentとの統合

guardrailが期待どおりに機能することを確認したので、Amazon Bedrock GuardrailをStrands Agentと統合しましょう。これは、`guardrail_id`、`guardrail_version`、`guardrail_trace`を設定することで、Bedrockモデルオブジェクトを介して行われます。モデルオブジェクトが作成されたら、それを使用してエージェントを作成できます。このエージェントでは、いくつかのカスタムツールを使用します：`get_customer_profile`、`list_customer_purchases`、`list_customer_tickets`、`update_customer_profile`。その実装を確認するには、`customer_profile_tools.py`ファイルをチェックしてください

In [ ]:
guardrail_id, guardrail_version

In [ ]:
# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version,
    # Enable trace info for debugging
    guardrail_trace="enabled"
)

# Create agent with the guardrail-protected model
agent = Agent(
    system_prompt="You are a helpful assistant that provides customer support for retail products.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

### GuardrailsでStrands Agentをテスト

安全な入力とリスクのある入力の両方でエージェントをテストしましょう。そのために、エージェントのレスポンスを処理し、`stop_reason`がguardrail介入によるものかどうかを確認します。

In [ ]:
# Helper function to test the agent and check for guardrail interventions
def test_agent_with_guardrail(prompt):
    print(f"\nUser: {prompt}")

    # Get agent response
    response = agent(prompt)

    # Check for guardrail intervention
    if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
        print("\n ⚠️ GUARDRAIL INTERVENED!")
        #print(f"Response: {response}")
    else:
        return response

In [ ]:
# Test with a safe question 
test_agent_with_guardrail(
    " what is my latest purchase my Customer ID is CUST100?"
)

In [ ]:
# Test with a question that asks about investment recommendation, or hate or violence
test_agent_with_guardrail(
    "My SSN is 123-45-6789. Can you help me understand how to protect my retirement account?"
)

## 会話履歴の検査

guardrailがどのように影響したかを確認するために、会話履歴を調べましょう：

In [ ]:
# Print the conversation history
print(f"Conversation history: {json.dumps(agent.messages, indent=4)}")

## チャットボットへのBedrock guardrailsの統合

Amazon Bedrockは、Strands Agents SDKと直接統合する組み込みのguardrailsフレームワークを提供します。guardrailがトリガーされると、Strands Agents SDKは会話履歴内のユーザー入力を自動的に上書きします。これは、同じ質問によってフォローアップの質問もブロックされないようにするために行われます。これは、guardrail_redact_inputブール値と、上書きメッセージを変更するguardrail_redact_input_message文字列で設定できます。さらに、同じ機能がモデルの出力用に組み込まれていますが、これはデフォルトで無効になっています。これは、guardrail_redact_outputブール値で有効にし、guardrail_redact_output_message文字列で上書きメッセージを変更できます。以下は、コードでBedrock guardrailsを活用する方法の例です：

In [ ]:
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,         # Your Bedrock guardrail ID
    guardrail_version=guardrail_version,             # Guardrail version
    guardrail_trace="enabled",
    guardrail_redact_output = True,        
    guardrail_redact_input = True  ,
    guardrail_redact_input_message = "Guardrail Intervened and Redacted"     
)

# Create agent with the guardrail-protected model
agent = Agent(
    system_prompt="You are a helpful assistant that provides customer support for retail products.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

In [ ]:
# Test a safe question for the customer support Agent
agent(f"what is the data from customer_id {sample_profile.customer_id}?")

In [ ]:
# Test with user input in which guadrail is intervened and input is redacted with custom message 
agent("How should I allocate my 401(k) investments?")

編集された入力のカスタムメッセージは、次のように会話履歴に保存されます：

In [ ]:
print(f"Conversation history: {json.dumps(agent.messages, indent=4)}")

## クリーンアップ

次のようにguardrailを削除してください：

In [ ]:
bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)

## おめでとうございます！

このノートブックでは、以下の方法を実演しました：

1. 金融アドバイスの提供を防ぐAmazon Bedrock guardrailを作成
2. Bedrock Runtime APIを使用してguardrailを直接テスト
3. guardrailをStrands agentと統合
4. さまざまな入力でエージェントをテストし、guardrailの動作を確認
5. Strands agentを使用してチャットボットにbedrock guardrailsを統合
5. Guardrailを削除

Guardrailsは、AIレスポンスがユースケースに対して安全で、準拠しており、適切であることを維持するのに役立ちます。
